In [2]:
import os

In [5]:
working_dir = os.path.dirname(os.getcwd())
os.chdir(working_dir)

In [6]:
import numpy as np
import pandas as pd
import geopandas as gpd

d:\Application\Conda\envs\geoneuro\lib\site-packages\geopandas\_compat.py:153: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  set_use_pygeos()
C:\Users\Cover\AppData\Local\Temp\ipykernel_102036\1960879352.py:3: DeprecationWarning: GeoPandas is set to use PyGEOS over Shapely. PyGEOS support is deprecatedand will be removed in GeoPandas 1.0, released in the Q1 of 2024. Please migrate to Shapely 2.0 (https://geopandas.org/en/stable/docs/user_guide/pygeos_to_shapely.html).
  import geopandas as gpd


## Read the dataset

In [7]:
sourcefile = 'chain_pro_new_plus_tc.csv'
readpath = './data/'
writepath = './data/'

In [8]:
df = pd.read_csv(readpath + sourcefile)
df.t_start, df.t_end = pd.to_datetime(df.t_start), pd.to_datetime(df.t_end)

In [9]:
df

,pid,t_start,t_end,poi_id,longitude,latitude,X,Y,ptype
0,1102234,2023-03-01 00:13:19,2023-03-01 07:29:20,0,114.026693,22.650468,194414.951879,2.507883e+06,1
1,1102234,2023-03-01 07:40:33,2023-03-01 22:24:33,1,114.054908,22.690259,197403.987064,2.512234e+06,2
2,1102234,2023-03-01 22:37:39,2023-03-01 22:42:41,4,114.031448,22.653732,194911.187574,2.508235e+06,0
3,1102234,2023-03-02 07:06:49,2023-03-02 07:29:38,17,114.025040,22.655869,194256.862442,2.508485e+06,0
4,1102234,2023-03-02 07:44:11,2023-03-02 22:09:46,1,114.054908,22.690259,197403.987064,2.512234e+06,2
...,...,...,...,...,...,...,...,...,...
7644,102181433,2023-12-22 18:39:10,2023-12-22 19:08:35,7,113.967361,22.586105,188167.740968,2.500875e+06,0
7645,102181433,2023-12-22 19:16:44,2023-12-22 23:28:44,4,113.992081,22.593155,190726.937740,2.501604e+06,0
7646,102181433,2023-12-22 23:29:04,2023-12-25 14:39:53,0,113.973390,22.594425,188806.743693,2.501784e+06,1
7647,102181433,2023-12-25 15:04:38,2023-12-31 21:31:20,0,113.973390,22.594425,188806.743693,2.501784e+06,1


In [10]:
def split_by_midnight(row):
    row = row.copy()
    start = row.t_start
    end = row.t_end
    if start.date() == end.date():
        rows = [row]
    else:
        # compute the date difference in days
        timedelta = end.date() - start.date()
        count = timedelta.days + 1
        date_range = pd.date_range(start.date(), end.date(), freq='D')[1:]
        vconcat = pd.concat([row] * count, axis=1).T
        vconcat.t_start = [start] + list(date_range)
        vconcat.t_end = list(date_range) + [end]
        # dataframe to list of rows
        rows = [r for _, r in vconcat.iterrows()]
    return rows

ans = df.apply(split_by_midnight, axis=1)
df = pd.concat(ans.explode().tolist(), axis=1).T.reset_index(drop=True)

In [11]:
df

,pid,t_start,t_end,poi_id,longitude,latitude,X,Y,ptype
0,1102234,2023-03-01 00:13:19,2023-03-01 07:29:20,0,114.026693,22.650468,194414.951879,2507882.981005,1
1,1102234,2023-03-01 07:40:33,2023-03-01 22:24:33,1,114.054908,22.690259,197403.987064,2512233.914594,2
2,1102234,2023-03-01 22:37:39,2023-03-01 22:42:41,4,114.031448,22.653732,194911.187574,2508234.840172,0
3,1102234,2023-03-02 07:06:49,2023-03-02 07:29:38,17,114.02504,22.655869,194256.862442,2508484.714036,0
4,1102234,2023-03-02 07:44:11,2023-03-02 22:09:46,1,114.054908,22.690259,197403.987064,2512233.914594,2
...,...,...,...,...,...,...,...,...,...
11258,102181433,2023-12-28 00:00:00,2023-12-29 00:00:00,0,113.97339,22.594425,188806.743693,2501783.83026,1
11259,102181433,2023-12-29 00:00:00,2023-12-30 00:00:00,0,113.97339,22.594425,188806.743693,2501783.83026,1
11260,102181433,2023-12-30 00:00:00,2023-12-31 00:00:00,0,113.97339,22.594425,188806.743693,2501783.83026,1
11261,102181433,2023-12-31 00:00:00,2023-12-31 21:31:20,0,113.97339,22.594425,188806.743693,2501783.83026,1


In [12]:
grids = gpd.read_file(readpath + 'ss_city_grid/ss_city_grid_by_cover.shp')
# lower the column name of the grids
grids.columns = grids.columns.str.lower()
# convert the fnid as the int
grids.fnid = grids.fnid.astype(int)
grids.chess_x = grids.chess_x.astype(int)
grids.chess_y = grids.chess_y.astype(int)

In [13]:
act_geo = gpd.GeoDataFrame(df.copy(), geometry=gpd.points_from_xy(df.longitude, df.latitude), crs='EPSG:4326')
act_geo_match_grids = gpd.sjoin_nearest(act_geo, grids, how='left', distance_col='distance')

act_geo_match_grids['distance'].max() * 111 * 1e3

d:\Application\Conda\envs\geoneuro\lib\site-packages\geopandas\array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


164.76565968757484

In [14]:
act_match = pd.DataFrame(act_geo_match_grids.drop(columns=['index_right', 'distance', 'geometry']))

## Landuse feature

In [15]:
landuse = gpd.read_file('./data/landuse/landuse_adjust.shp')

In [16]:
landuse[['XHDLBM', 'XHDLMC']].drop_duplicates().sort_values(by='XHDLBM').set_index('XHDLBM')

,XHDLMC
XHDLBM,
011,水田
012,水浇地
013,旱地
021,果园
022,茶园
023,其他园地
031,有林地
032,灌木林地
033,其他林地


In [17]:
def type_mapping(id):
    idnumber = int(id)
    if idnumber in [51, 52, 54]:
        return 'Consumption' 
    elif idnumber in [53]:
        return 'Business'
    elif 61 <= idnumber <= 63:
        return 'Industry'
    elif idnumber in [71]:
        return 'Residence'
    elif idnumber in [81, 82, 86, 107]: 
        return 'Public'
    elif idnumber in [91, 93, 95]:
        return 'Special'
    elif idnumber in [83]:
        return 'Science & Education'
    elif idnumber in [84]:
        return 'Medical'
    elif idnumber in [85, 94]:
        return 'Culture'
    elif idnumber in [87, 88]:
        return 'Park & Scenery'
    elif idnumber in [101, 105, 106]:
        return 'Transportation'
    elif idnumber in [102, 103]:
        return 'City Road'
    else:
        return 'Wild'
landuse.XHDLBM.map(type_mapping).value_counts()

XHDLBM
Wild                   64555
Industry               19596
City Road              11757
Residence              10537
Public                  9175
Consumption             5253
Science & Education     2048
Park & Scenery          1610
Culture                 1549
Business                1391
Transportation          1201
Medical                  576
Special                  556
Name: count, dtype: int64

In [18]:
city_grid_buffer_utm = grids.to_crs("EPSG:32650").centroid.buffer(250)
city_grid_buffer_utm = gpd.GeoDataFrame(data=grids.drop(columns=['geometry']), geometry=city_grid_buffer_utm).reset_index()

In [19]:
landuse_simple = landuse[['OBJECTID', 'XHDLBM', 'geometry']].to_crs("EPSG:32650").copy()
landuse_simple['usetype'] = landuse_simple['XHDLBM'].map(type_mapping)

In [20]:
buffer_intersect = city_grid_buffer_utm.overlay(landuse_simple, how='intersection')

In [21]:
buffer_intersect['area'] = buffer_intersect.geometry.area
buffer_usetype = buffer_intersect.groupby(['fnid', 'usetype'])['area'].sum().unstack().fillna(0)

In [22]:
TF = buffer_usetype.values / np.sum(buffer_usetype.values, axis=1).reshape(-1, 1)

In [23]:
ans = landuse_simple[['usetype', 'geometry']].copy()
ans['area'] = ans.geometry.area
type_area = ans.groupby('usetype')['area'].sum()
IWF = np.log(np.sum(type_area.values) / type_area.values)

In [24]:
buffer_usetype_uniform = pd.DataFrame(TF * IWF, columns=buffer_usetype.columns.map(lambda s: ('LU_' + s).replace(' ', '_')), index=buffer_usetype.index)

In [25]:
city_grid_with_LU = pd.merge(grids, buffer_usetype_uniform, how='left', left_on='fnid', right_index=True)

In [26]:
city_grid_with_LU

,fnid,chess_x,chess_y,geometry,LU_Business,LU_City_Road,LU_Consumption,LU_Culture,LU_Industry,LU_Medical,LU_Park_&_Scenery,LU_Public,LU_Residence,LU_Science_&_Education,LU_Special,LU_Transportation,LU_Wild
0,413,56,1,"POLYGON ((113.88582 22.44962, 113.88582 22.451...",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,3.99794,0.000000
1,665,308,1,"POLYGON ((114.50078 22.44962, 114.50078 22.451...",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.516393
2,666,309,1,"POLYGON ((114.50322 22.44962, 114.50322 22.451...",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.516393
3,667,310,1,"POLYGON ((114.50566 22.44962, 114.50566 22.451...",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.516393
4,668,311,1,"POLYGON ((114.50810 22.44962, 114.50810 22.451...",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.516393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31473,65743,55,184,"POLYGON ((113.88338 22.85514, 113.88338 22.857...",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.516393
31474,65745,57,184,"POLYGON ((113.88826 22.85514, 113.88826 22.857...",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.516393
31475,66100,55,185,"POLYGON ((113.88338 22.85735, 113.88338 22.859...",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.516393
31476,66101,56,185,"POLYGON ((113.88582 22.85735, 113.88582 22.859...",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.516393


In [34]:
featureset = pd.merge(act_match.drop(columns=['chess_x', 'chess_y']), city_grid_with_LU, how='left', on='fnid')

In [35]:
featureset.drop(columns=['geometry'], inplace=True)

In [36]:
featureset.to_csv(writepath + 'featureset.csv', index=False)

In [45]:
be = pd.read_csv(readpath + 'featureset.csv')

In [46]:
be.rename(columns={'pid': 'who'}, inplace=True)
be = be.assign(t_start = pd.to_datetime(be['t_start']),
          t_end = pd.to_datetime(be['t_end']))

In [47]:
be['date'] = be.t_start.dt.date

In [49]:
be = be.groupby(['who', 'date'], as_index=False).apply(lambda df: df.sort_values('t_start'))
be['seiqd'] = be.groupby(['who', 'date']).cumcount()
be.reset_index(drop=True)

In [52]:
columns = ['who','date', 'seiqd', 't_start', 't_end', 'poi_id', 'longitude', 'latitude', 'X', 'Y',
       'ptype', 'fnid', 'chess_x', 'chess_y', 'LU_Business', 'LU_City_Road',
       'LU_Consumption', 'LU_Culture', 'LU_Industry', 'LU_Medical',
       'LU_Park_&_Scenery', 'LU_Public', 'LU_Residence',
       'LU_Science_&_Education', 'LU_Special', 'LU_Transportation', 'LU_Wild']

be = be.reindex(columns=columns)
# convert the date to integer as %Y%m%d format
be['date'] = be['date'].apply(lambda x: int(x.strftime('%Y%m%d')))

In [53]:
be.to_csv(readpath + 'featureset.csv', index=False)  

## Sketch Pad